In [1]:
DATA = "data"

In [2]:
import os

In [3]:
import spacy.cli
nlp = spacy.load('it_core_news_sm')
nlp_multi = spacy.load('xx_ent_wiki_sm')

In [4]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /home/toccasana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
songs = os.listdir(DATA)

In [6]:
data = []

In [7]:
def get_grams(text):
    toks = word_tokenize(text)
    bigs = [a + "_" + b for a,b in nltk.bigrams(toks)]
    return toks + bigs

In [8]:
import csv
import re
from gensim.models.doc2vec import TaggedDocument
data = list()
for song in songs:
    with open(os.path.join(DATA,song),"r",encoding='utf-8') as file:
        title,artist = song[:-4].split("_",1)
        lyrics = get_grams(file.read().lower())
        data.append(TaggedDocument(lyrics ,(title.strip(), artist.strip())))

In [9]:
data[0]

TaggedDocument(words=['going', 'back', 'to', 'the', 'corner', 'where', 'i', 'first', 'saw', 'you', 'gon', 'na', 'camp', 'in', 'my', 'sleeping', 'bag', ',', 'i', "'m", 'not', 'gon', 'na', 'move', 'got', 'some', 'words', 'on', 'cardboard', ',', 'got', 'your', 'picture', 'in', 'my', 'hand', 'saying', ',', '``', 'if', 'you', 'see', 'this', 'girl', ',', 'can', 'you', 'tell', 'her', 'where', 'i', 'am', '?', "''", 'some', 'try', 'to', 'hand', 'me', 'money', ',', 'they', 'do', "n't", 'understand', 'i', "'m", 'not', 'broke', ',', 'i', "'m", 'just', 'a', 'broken-hearted', 'man', 'i', 'know', 'it', 'makes', 'no', 'sense', ',', 'what', 'else', 'can', 'i', 'do', '?', 'and', 'how', 'can', 'i', 'move', 'on', 'when', 'i', "'m", 'still', 'in', 'love', 'with', 'you', '?', "'cause", 'if', 'one', 'day', 'you', 'wake', 'up', ',', 'and', 'find', 'that', 'you', "'re", 'missing', 'me', 'and', 'your', 'heart', 'starts', 'to', 'wonder', 'where', 'on', 'this', 'earth', 'i', 'could', 'be', 'thinking', 'maybe', 'y

In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
from gensim.models import Doc2Vec
model = Doc2Vec(data, vector_size=100, window=10, min_count=5,workers=8)

2021-05-21 14:24:36,703 : INFO : collecting all words and their counts
2021-05-21 14:24:36,704 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-05-21 14:24:38,339 : INFO : collected 767740 word types and 1635 unique tags from a corpus of 1098 examples and 4934402 words
2021-05-21 14:24:38,340 : INFO : Loading a fresh vocabulary
2021-05-21 14:24:39,034 : INFO : effective_min_count=5 retains 92000 unique words (11% of original 767740, drops 675740)
2021-05-21 14:24:39,035 : INFO : effective_min_count=5 leaves 3933321 word corpus (79% of original 4934402, drops 1001081)
2021-05-21 14:24:39,342 : INFO : deleting the raw counts dictionary of 767740 items
2021-05-21 14:24:39,355 : INFO : sample=0.001 downsamples 33 most-common words
2021-05-21 14:24:39,356 : INFO : downsampling leaves estimated 3459916 word corpus (88.0% of prior 3933321)
2021-05-21 14:24:39,660 : INFO : estimated required memory for 92000 words and 100 dimensions: 120581000 bytes
2021-05-

In [17]:
model.wv.most_similar('amore')

[('aspetto', 0.9910622239112854),
 ('sempre', 0.9908531904220581),
 ('cuore', 0.9896554350852966),
 ('tutto', 0.9881945252418518),
 ('così', 0.9880496263504028),
 ('siamo', 0.9879838228225708),
 ('mare', 0.9859680533409119),
 ('senza', 0.9855455160140991),
 ('sento', 0.9852760434150696),
 ('ora', 0.9851750135421753)]

In [13]:
from gensim.scripts.word2vec2tensor import word2vec2tensor

In [14]:
help(word2vec2tensor)

Help on function word2vec2tensor in module gensim.scripts.word2vec2tensor:

word2vec2tensor(word2vec_model_path, tensor_filename, binary=False)
    Convert file in Word2Vec format and writes two files 2D tensor TSV file.
    
    File "tensor_filename"_tensor.tsv contains word-vectors, "tensor_filename"_metadata.tsv contains words.
    
    Parameters
    ----------
    word2vec_model_path : str
        Path to file in Word2Vec format.
    tensor_filename : str
        Prefix for output files.
    binary : bool, optional
        True if input file in binary format.



In [15]:
help(model.save)

Help on method save in module gensim.models.base_any2vec:

save(fname_or_handle, **kwargs) method of gensim.models.doc2vec.Doc2Vec instance
    "Save the object to file.
    
    Parameters
    ----------
    fname_or_handle : {str, file-like object}
        Path to file where the model will be persisted.
    **kwargs : object
        Key word arguments propagated to :meth:`~gensim.utils.SaveLoad.save`.
    
    See Also
    --------
    :meth:`~gensim.models.base_any2vec.BaseAny2VecModel.load`
        Method for load model after current method.



In [41]:
#model.save("test.bin")

2021-05-18 22:27:43,966 : INFO : saving Doc2Vec object under test.bin, separately None
2021-05-18 22:27:45,090 : INFO : saved test.bin


In [21]:
model.save_word2vec_format("aaa.model", doctag_vec=True,word_vec=False)

2021-05-21 14:19:38,462 : INFO : storing 1635x100 projection weights into aaa.model


In [ ]:
word2vec2tensor("aaa.model", "tenss2.tsv")

In [20]:
model.wv["amore"]

array([-3.0414110e-01,  8.9024329e-01,  4.6885315e-01, -3.8037524e-01,
       -8.2630616e-01, -1.5841106e-01,  6.1700702e-01, -1.0527494e+00,
        1.3500911e+00,  3.9530137e-01,  6.6823167e-01,  2.2794807e-01,
       -3.0136892e-01,  2.0185579e-01, -2.1184696e-01, -2.6370502e-01,
       -3.7599313e-01, -5.1218569e-01,  1.9673681e-01,  1.0049195e-01,
        8.2977551e-01, -1.4552003e-01,  8.7121040e-02, -9.6745294e-01,
        2.9916680e-01,  4.7947526e-01, -5.7060635e-01, -2.3023169e+00,
        6.6403449e-01, -8.1064567e-02,  3.5554737e-01,  1.4116858e-01,
       -1.3702829e+00,  1.0601436e+00, -2.4336295e-01, -6.7433542e-01,
       -2.9505101e-01,  1.5755191e-01, -1.6761215e-01, -7.4095881e-01,
       -2.0910585e-01, -6.6895312e-01,  8.6308336e-03,  3.5038507e-01,
        7.6972431e-01, -1.7457867e+00, -1.5643402e+00, -4.1801029e-01,
       -8.5821819e-01,  3.0467752e-01, -2.2392602e-01,  6.5956187e-01,
       -6.8747841e-02, -7.5739169e-01, -1.3386175e-01,  2.8143838e-01,
      

In [24]:
import gensim

In [25]:
gensim.__version__

'3.8.3'

In [29]:
model.docvecs.most_similar("Eminem")

[("The Man Who Can't Be Moved", 0.9732003211975098),
 ('Crack A Bottle', 0.9659927487373352),
 ('Red Hot Chili Peppers', 0.9595954418182373),
 ('Bruce Springsteen', 0.9595569372177124),
 ('Coldplay', 0.9552341103553772),
 ('Viva La Vida', 0.9543634653091431),
 ('Wind It Up', 0.9521149396896362),
 ("Cupid's Chokehold  Breakfast in America", 0.9503450989723206),
 ('Gym Class Heroes', 0.950168251991272),
 ('Duffy', 0.9496716260910034)]

In [35]:
model.docvecs.save_word2vec_format("bbb.model")

2021-05-21 15:26:47,559 : INFO : storing 1635x100 projection weights into bbb.model


In [37]:
word2vec2tensor("bbb.model", "tenss2.tsv",binary=True)

2021-05-21 15:27:08,195 : INFO : loading projection weights from bbb.model
2021-05-21 15:27:08,201 : WARNING : duplicate word '' in word2vec file, ignoring all but first
2021-05-21 15:27:08,203 : WARNING : duplicate word '' in word2vec file, ignoring all but first
2021-05-21 15:27:08,204 : WARNING : duplicate word '' in word2vec file, ignoring all but first
2021-05-21 15:27:08,206 : WARNING : duplicate word '0' in word2vec file, ignoring all but first
2021-05-21 15:27:08,207 : WARNING : duplicate word '' in word2vec file, ignoring all but first
2021-05-21 15:27:08,208 : WARNING : duplicate word '8' in word2vec file, ignoring all but first
2021-05-21 15:27:08,210 : WARNING : duplicate word '' in word2vec file, ignoring all but first
2021-05-21 15:27:08,211 : WARNING : duplicate word '' in word2vec file, ignoring all but first
2021-05-21 15:27:08,212 : WARNING : duplicate word '6' in word2vec file, ignoring all but first
2021-05-21 15:27:08,212 : WARNING : duplicate word '' in word2vec f

EOFError: unexpected end of input; is count incorrect or file otherwise damaged?